In [1]:
import torch
import numpy as np
import copy
import matplotlib.pyplot as plt

from torch.utils.data import Dataset

from utilities import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
# batch_size = 20
batch_size = 128

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

In [4]:
args = dict()

args['dim_z'] = 5

In [5]:
X_train = train_data.data
X_train_left = X_train[:,:,:14]
X_train_right = X_train[:,:,14:]
Y_train = train_data.targets
X_test = test_data.data
X_test_left = X_test[:,:,:14]
X_test_right = X_test[:,:,14:]
Y_test = test_data.targets

In [6]:
X_train_left = X_train_left.numpy().reshape(len(X_train),-1)
X_train_right = X_train_right.numpy().reshape(len(X_train),-1)
X_test_left = X_test_left.numpy().reshape(len(X_test),-1)
X_test_right = X_test_right.numpy().reshape(len(X_test),-1)

In [7]:
X_train_data = list(zip(torch.Tensor(X_train_left).to(device),torch.Tensor(X_train_right).to(device)))
X_test_data = list(zip(torch.Tensor(X_test_left).to(device),torch.Tensor(X_test_right).to(device)))

In [8]:
X_train_loader = torch.utils.data.DataLoader(X_train_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=True, shuffle=True)
X_test_loader = torch.utils.data.DataLoader(X_test_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=True, shuffle=False)

In [23]:
class BilinearNWJNCE(nn.Module):
    def __init__(self,
         critic: nn.Module, 
         args: Optional[Dict] = None,
         cuda: Optional[int] = None) -> None:
        
        super(BilinearNWJNCE, self).__init__()
        self.critic = critic
        self.criterion = torch.nn.CrossEntropyLoss(reduction='none')
    
    def forward(self, x, y, y0=None, K=None):
        
        '''
        x:    n x p
        y:    n x d true
        y0:   n x d fake 
        '''

        output  = self.PMI(x,y)
#         output = torch.clamp(output,-5,15)
        return output.mean()
    
    def MI(self, x, y, K=10):
        mi = 0
        for k in range(K):
            y0 = y[torch.randperm(y.size()[0])]
            mi += self.forward(x,y,y0)
            
        return -mi/K   
    
    def PMI(self, x, y, y0=None, K=None):
        '''
        x:    n x p
        y:    n x d true
        y0:   n x d fake 
        '''
        gxy = self.critic(x,y)
        if isinstance(gxy, tuple):
            hx,hy = gxy
            similarity_matrix = hx @ hy.t()
            
            pos_mask = torch.eye(hx.size(0),dtype=torch.bool)
            g = similarity_matrix[pos_mask].view(hx.size(0),-1)
            g0 = similarity_matrix[~pos_mask].view(hx.size(0),-1)
            
            output = torch.exp(torch.logsumexp(g0,1).view(-1,1)-1.)/torch.Tensor([hx.size(0)]).to(device)-g.view(-1,1)


        else:      
            output = None
                 
        return output

In [24]:
class BilinearCritic(nn.Module):
    
    '''
    encoder_x : dx -> feature_dim
    encoder_y : dy -> feature_dim
    u_func : 2*feature_dim -> 1
    '''
    
    def __init__(self,
                 encoder_x: nn.Module,
                 encoder_y: nn.Module,
                 u_func: Optional[nn.Module] = None,
                 tau: Optional[float] = 1.):
        
        super(BilinearCritic,self).__init__()
        self.encoder_x = encoder_x
        self.encoder_y = encoder_y
        self.u_func = u_func
        self.log_tau = torch.nn.Parameter(torch.Tensor([np.log(tau)]))
        
    
    def forward(self, x, y, tau=None):
        if tau is None:
            tau = torch.exp(self.log_tau)
        tau = torch.sqrt(tau)
        hx = self.norm(self.encoder_x(x))
        hy = self.norm(self.encoder_y(y))
        if self.u_func is not None:
            u = self.u_func(hx,hy)
            return hx/tau, hy/tau, u  
        
        return hx/tau, hy/tau  
    
    def norm(self,z):
        return torch.nn.functional.normalize(z,dim=1)
        #return z/torch.sqrt(torch.square(z).sum(1).view(-1,1))
        #return z

In [25]:
class Wrapper(nn.Module):
    def __init__(self, func):
        super(Wrapper,self).__init__()
        self.func = func
#         self.dx = dx
#         self.dy = dy
        
    def forward(self, x, y):
        xy = torch.cat([x,y],dim=1)
        
        return self.func(xy)

In [26]:
lam = 1
p = args['dim_z']
# args = {}
args['lr'] = 1e-3
args['latent_dim'] = 100
args['num_epochs'] = 50
args["input_dim"] = 2*p
#args['batch_size']=50
feature_dim = 512
encoder_x = MLP(p, output_dim=feature_dim)
encoder_y = MLP(p, output_dim=feature_dim)
# u_func = Wrapper(MLP(2*feature_dim,hidden_dim=[128]))
critic = BilinearCritic(encoder_x, encoder_y)
#critic = Wrapper(MLP(args["input_dim"],2))
# u_func = Wrapper(MLP(args["input_dim"]))
mi_model = BilinearNWJNCE(critic)
#mitrainer = FenchelInfoNCETrainer(mi_model)

mi_model = mi_model.to(device)

In [27]:
encoder_left = MLP(14*28, args['dim_z'])
encoder_right = MLP(14*28, args['dim_z'])

encoder_left = encoder_left.to(device)
encoder_right = encoder_right.to(device)

In [28]:
optimizer = torch.optim.Adam([*encoder_left.parameters(),
            *encoder_right.parameters(),*mi_model.parameters()],lr=1e-4)

In [29]:
# number of epochs to train the model
n_epochs = 50  # suggest training between 20-50 epochs
mi_model.train() # prep model for training

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    train_mi = 0.0
    ###################
    # train the model #
    ###################
#     model.train()
    mi_model.train()
    for images_left, images_right in X_train_loader:
        # load the (un)labeled data
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        z_left = encoder_left(images_left)
        z_right = encoder_right(images_right)
        mi_loss = mi_model(z_left,z_right,None)
        
        # forward pass: compute predicted outputs by passing inputs to the model
#         output = model(data)
        # calculate the loss
        total_loss = mi_loss
        # backward pass: compute gradient of the loss with respect to model parameters
        total_loss.backward()
        optimizer.step()
        # perform a single optimization step (parameter update)
        
        # update running training loss
        # train_loss += loss.item()*data.size(0)
        train_mi += -mi_loss.item()*images_left.size(0)
        
    train_mi = train_mi/len(X_train_loader.dataset)
#     print('Epoch: {}\t MI: {:4f}'.format(epoch+1, train_mi))
    print('Epoch: {}\t MI: {:4f}, tau: {:4f}'.format(epoch+1, train_mi, torch.exp(critic.log_tau).detach().cpu().numpy()[0]))

Epoch: 1	 MI: 0.366498, tau: 0.943648
Epoch: 2	 MI: 0.531452, tau: 0.895418
Epoch: 3	 MI: 0.602289, tau: 0.849882
Epoch: 4	 MI: 0.688108, tau: 0.805749
Epoch: 5	 MI: 0.762779, tau: 0.764039
Epoch: 6	 MI: 0.831783, tau: 0.725002
Epoch: 7	 MI: 0.898371, tau: 0.688415
Epoch: 8	 MI: 0.964879, tau: 0.653965
Epoch: 9	 MI: 1.037014, tau: 0.621187
Epoch: 10	 MI: 1.113843, tau: 0.589889
Epoch: 11	 MI: 1.196592, tau: 0.560077
Epoch: 12	 MI: 1.281249, tau: 0.531858
Epoch: 13	 MI: 1.366294, tau: 0.505234
Epoch: 14	 MI: 1.455714, tau: 0.480017
Epoch: 15	 MI: 1.549686, tau: 0.456087
Epoch: 16	 MI: 1.650456, tau: 0.433344
Epoch: 17	 MI: 1.750597, tau: 0.411876
Epoch: 18	 MI: 1.855595, tau: 0.391541
Epoch: 19	 MI: 1.964370, tau: 0.372238
Epoch: 20	 MI: 2.079774, tau: 0.353920
Epoch: 21	 MI: 2.194512, tau: 0.336603
Epoch: 22	 MI: 2.317015, tau: 0.320190
Epoch: 23	 MI: 2.444550, tau: 0.304580
Epoch: 24	 MI: 2.580357, tau: 0.289716
Epoch: 25	 MI: 2.714186, tau: 0.275697
Epoch: 26	 MI: 2.853898, tau: 0.26

In [ ]:
# critic.tau
# critic.tau
# critic.tau.detach().cpu().numpy()

In [30]:
X_train_loader = torch.utils.data.DataLoader(X_train_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=False, shuffle=False)
X_test_loader = torch.utils.data.DataLoader(X_test_data, batch_size=batch_size, 
    num_workers=num_workers, drop_last=False, shuffle=False)

In [31]:
encoder_left.eval()
encoder_right.eval()

Z_train = []
Z_test = []

for images_left, images_right in X_train_loader:
    z_left = encoder_left(images_left).detach().cpu().numpy()
    z_right = encoder_right(images_right).detach().cpu().numpy()
    z = np.concatenate([z_left,z_right],1)
    Z_train.append(z)

for images_left, images_right in X_test_loader:
    z_left = encoder_left(images_left).detach().cpu().numpy()
    z_right = encoder_right(images_right).detach().cpu().numpy()
    z = np.concatenate([z_left,z_right],1)
    Z_test.append(z)

Z_train = np.concatenate(Z_train,0)
Z_test = np.concatenate(Z_test,0)

In [32]:
data_dict = dict()
data_dict['Z_train'] = Z_train
data_dict['Y_train'] = Y_train
data_dict['Z_test'] = Z_test
data_dict['Y_test'] = Y_test
data_dict['method'] = 'NWJ'

In [33]:
np.save('features/nwj_d_5.npy',data_dict)